<a href="https://colab.research.google.com/github/MaricelaMH/SIMULACION-II/blob/main/Frecuentistas_vs_Bayesianos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # F R E C U E N T I S T A S   _  V S  _  B A Y E S I A N O S

 Principalmente el desacuerdo de los frecuentistas con los bayesianos es la definición de probabilidad.

1. **Definición frecuentista de la probabilidad:** Sostiene que la
probabilidad de un evento es el límite al que tiende la frecuencia relativa de ocurrencia de dicho evento cuando el número de ensayos se incrementa indefinidamente. Es decir, según los frecuentistas, la probabilidad se basa en la observación empírica de eventos repetidos bajo condiciones similares.

* Si realizamos un experimento muchas veces, la probabilidad de un evento $A$ se define como el valor al que tiende la proporción de veces que ocurre $A$ en relación al número total de ensayos.

* Matemáticamente, se expresa como:

\\
 $$ P(A) = \lim_{n \to ∞ } ( \frac{\text{Número de veces que ocurre el evento $A$}}{\text{Número total de repeticiones del experimento}} )$$

\\
2. **Definición bayesiana de la probabilidad:** La probabilidad en el enfoque bayesiano se basa en el teorema de Bayes, que establece cómo actualizar las creencias o probabilidades iniciales (llamadas probabilidades a priori) cuando se adquieren nuevos datos o evidencia, resultando en una nueva probabilidad a posteriori.

* El teorema de Bayes se expresa matemáticamente como:

$$P(A|B)=\frac{P(B|A)* P(A)}{P(B)}$$

Donde:

* $P(A∣B)$ es la probabilidad de $A$ dado que se ha observado $B$ (la probabilidad a posteriori).

* $P(B∣A)$ es la probabilidad de observar $B$ si $A$ es verdadero.

* $P(A)$ es la probabilidad de $A$ antes de observar $B$ (la probabilidad a priori)

* $P(B)$ es la probabilidad total de observar $B$.

Esta diferencia que podría decirse que es sutil, puede llevar, en la práctica, a enfoques muy diferentes para el análisis estadístico de datos. A continuación, exploraremos algunos ejemplos elegidos para ilustrar las diferencias de enfoque, junto con el código Python asociado para demostrar los aspectos prácticos de los enfoques frecuentista y bayesiano.



# Ejemplo 1: Mediciones del flujo de fotones
Apuntamos un telescopio al cielo y observamos la luz que proviene de una sola estrella. Para simplificar, supondremos que el flujo fotónico real de la estrella es constante con el tiempo, es decir ,que tiene un valor fijo $F$, también ignoraremos efectos como los errores sistemáticos del fondo del cielo. Supondremos que se realizan una serie de $N$ mediciones, donde la i-ésima medición informa el flujo observado $F_i$ y el error $e_i$. La pregunta es dado este conjunto de mediciones $D = \{F_i, e_i\}$, ¿cuál es nuestra mejor estimación del flujo verdadero $F$?

* El articulo nos menciona primeramente que debemos de generar una muestra com media 1000 y en error definido como $e$:

In [1]:
import numpy as np
np.random.seed(2) # Si ejecutamos el código con la misma semilla (en este caso, 2), obtendremos los mismos números aleatorios

# Generamos muestras de números aleatorios de una distribución normal
e = np.random.normal(30, 3, 50) # Ponemos los datos de la media, desviacion estádar y tamaño de la muestra
F = np.random.normal(1000, e) # Generamos un array de 50 valores con media 1000 y desviación estándar e

* Ahora dadas las mediciones y errores anteriores , ¿cuál es nuestra mejor estimación puntual del flujo verdadero?, por lo que el articulo nos da dos enfoques de solución, el frecuentista y el bayesiano.

### -> Enfoque frecuentista para la medición del flujo
Por razones de simplicidad analítica y precisión numérica, la log-verosimilitud se utiliza para este fin, ya que maximizar la log-verosimilitud es equivalente a maximizar la verosimilitud, por lo que es más conveniente considerar la log-verosimilitud (transforma multiplicaciones de probabilidades en sumas) en lugar de la verosimilitud directa, por lo que esta biene definida como:

 $$ log \mathscr{L} (D|F) = -\frac{1}{2} \sum_{i=1}^{N} [ log(2\pi e_i^2)+\frac{(F_i - F)^2}{e_i^2}]$$

Nos gustaría determinar el valor de $F$ que maximiza la probabilidad. Para este problema simple, la maximización se puede calcular analíticamente, estableciendo (por ejemplo, estableciendo $\frac{d log \mathscr{L}}{dF}=0$), el resultado de esta maximización lleva a la siguiente estimación puntual de $F$:

$$\hat{F} = \frac{\sum{w_i F_i}}{\sum{w_i}}$$

Se toma cada valor en $e$ (la desviación estándar de cada elemento en $F$) y se calcula el peso correspondiente como

$$w_i = \frac{1}{e^2_i}$$

el uso de $e^2_i$ indica que los valores con una mayor desviación estándar ($e$) tendrán menor peso en el promedio ponderado.

De ogual manera podemos preguntarnos cuál es la incertidumbre de nuestra estimación. Una forma de lograrlo en el enfoque frecuentista es construir una aproximación gaussiana a la probabilidad máxima, por lo que se obtiene que:

$$\sigma_{\hat{F}}=(\sum_{i=1}^{N} w_i)^{-\frac{1}{2}}$$

lo implementamos de la siguiente manera:

In [2]:
# Calculamos los pesos inversamente proporcionales al cuadrado del error
w = 1. / e ** 2  # 1./ es una forma de dividir 1.0 por el valor que sigue
F_hat = np.sum(w * F) / np.sum(w)
sigma_F = w.sum() ** -0.5
print("El valor F que maximiza la probabilidad es: ",round(F_hat,2))
print("La incertidumbre asociada a la estimación es: ",round(sigma_F,2))

El valor F que maximiza la probabilidad es:  998.65
La incertidumbre asociada a la estimación es:  4.11


### -> Enfoque bayesiano para la medición del flujo

Como es de esperar, el enfoque bayesiano comienza y termina con probabilidades. El resultado fundamental de interés es nuestro conocimiento de los parámetros en cuestión, para calcular este resultado, aplicamos a continuación el teorema de Bayes, una ley fundamental de la probabilidad:

$$P(F|D)=\frac{P(D|F) P(F)}{P(D)}$$

donde:

* $P(F|D)$: es la probabilidad de los parámetros del modelo dados los datos.
* $P(D|F)$: que es proporcional a la $\mathscr{L} (D|F)$ utilizada en el enfoque frecuentista.
* $P(F)$: El modelo anterior, que codifica lo que sabíamos sobre el modelo antes de considerar los datos $D$.
* $P(D)$: La evidencia del modelo, que en la práctica equivale simplemente a un término de normalización.

su planteamiento es fundamentalmente contrario a la filosofia frecuentista, que dice que las probabilidades no tienen significado para parámetros fijos del modelo como F. Sin embargo, en la concepción bayesiana de la probabilidad esto no plantea ningún problema.
Es decir, con una distribución previa plana en F, la distribución posterior bayesiana se maximiza exactamente en el mismo valor que el resultado frecuentista. Por lo tanto, a pesar de las diferencias filosóficas, vemos que las estimaciones puntuales bayesianas y frecuentistas son equivalentes para este problema simple.

### -> Divergencia de resultados

En el ejemplo simple anterior, los enfoques frecuentista y bayesiano arrojan básicamente el mismo resultado. Sin embargo, si bien es fácil demostrar que los dos enfoques suelen ser equivalentes para problemas simples, también es cierto que pueden divergir en gran medida en otras situaciones, esta divergencia suele manifestarse de dos maneras diferentes:

1. El manejo de parámetros molestos: es decir, parámetros que afectan el resultado final, pero que no son de interés en ningún otro sentido.

Un parámetro molesto es cualquier cantidad cuyo valor no es directamente relevante para el objetivo de un análisis, pero que sin embargo, es necesario para determinar el resultado que interesa.
2. El diferente manejo de la incertidumbre: por ejemplo, la diferencia sutil (y a menudo pasada por alto) entre los intervalos de confianza frecuentistas y las regiones creíbles bayesianas.

A continuación discutiremos ejemplos de esto.

# Ejemplo 2: El juego de billar de Bayes

Se trata de un juego de apuestas en el que Alicia y Bob apuestan sobre el resultado de un proceso que no pueden observar directamente.
Alice y Bob entran en una habitación, detrás de una cortina hay una mesa de billar, que no pueden ver, su amiga Carol hace rodar una bola por la mesa y marca dónde cae. Una vez que la marca está en su lugar, Carol comienza a hacer rodar nuevas bolas por la mesa. Si la bola cae a la izquierda de la marca, Alice obtiene un punto; si cae a la derecha de la marca, Bob obtiene un punto. Podemos suponer que los lanzamientos de Carol son imparciales: es decir, las bolas tienen la misma probabilidad de terminar en cualquier lugar de la mesa. La primera persona que alcance los seis puntos gana el juego, en este caso, la ubicación de la marca (determinada por el primer lanzamiento) puede considerarse un parámetro molesto: es desconocido y no tiene interés inmediato, pero claramente debe tenerse en cuenta al predecir el resultado de los lanzamientos posteriores. Si el primer lanzamiento se establece muy a la derecha, los lanzamientos posteriores favorecerán a Alice. Si se establece muy a la izquierda, Bob será el favorecido. Con esta configuración, buscamos responder a esta pregunta: en un juego en particular, después de ocho lanzamientos, Alice tiene cinco puntos y Bob tiene tres puntos. ¿Cuál es la probabilidad de que Bob obtenga seis puntos y gane el juego?. Intuitivamente, nos damos cuenta de que, como Alice recibió cinco de los ocho puntos, la ubicación del marcador probablemente la favorezca. Dado que tiene tres oportunidades de obtener un sexto punto antes de que Bob pueda ganar, parece que lo ha conseguido. Pero, cuantitativamente hablando, ¿cuál es la probabilidad de que Bob persista para ganar?

### -> Un enfoque frecuentista ingenuo

Como cinco bolas de ocho calleron en el lado del marcador de Alicia, calculamos la estimación de máxima verosimilitud de p, dada por:

$$\hat{p} = \frac{5}{8}$$

De la probabilidad binomial se sigue un resultado de manera directa. Suponiendo esta probabilidad máxima, podemos calcular la probabilidad de que Bob gane, lo que requiere que obtenga un punto en cada uno de los tres lanzamientos siguientes. Esto viene dado por:

$$P(B)=(1-\hat{p})^3$$

lo implementamos de la siguiente manera

In [3]:
p_hat = 5/8
P_B = (1-p_hat)**3
print("La probabilidad de que Bob gane es: ",round(P_B,2))

La probabilidad de que Bob gane es:  0.05


### -> Enfoque bayesiano
Un enfoque bayesiano para este problema implica marginalizar (es decir, integrar) la $p$ desconocida de modo que, suponiendo que lo anterior sea preciso, nuestro resultado debe ser independiente de su valor real.
En este sentido, consideraremos las siguientes cantidades:
1. $B=$ Bob gana.
2. $D=$ Datos observados, es decir $D=(5,3)$
3. $\rho =$ probabilidad desconocida de que una pelota caiga en el lado de Alicia durante el juego actual.
Comenzamos aplicando la definición de probabilidad condicional para
expandir el término $P(B,\rho|D)$:

$$P(B|D)=\int P(B|\rho,D)P(\rho|D)d\rho$$

Finalmente, utilizando la misma identidad de probabilidad con la que comenzamos, podemos expandir $P(D)$ en el denominador para encontrar:

$$P(B|D)=\frac{\int P(B|\rho,D)P(D|\rho)P(\rho)d\rho}{\int P(D|\rho)P(\rho)d\rho}$$

donde:

* $P(B∣\rho,D):$ Es la probabilidad de que Bob gane tres veces seguidas dados un valor de $\rho$ (probabilidad de ganar un juego) y los datos $D$. Se calcula como $(1-\rho)^3$
* $P(D|\rho):$ Es la probabilidad de obtener exactamente 5 éxitos (victorias de Alice) en 8 intentos, dado el valor $\rho$. Esto se obtiene usando la distribución binomial:
$P(D|\rho)$ -> $\rho^5 (1-\rho)^3$
* $P(\rho):$ Es la probabilidad a priori de $\rho$. En este caso, se asume que $\rho$ es uniforme entre 0 y 1.

por lo que

$$P(B|D)=\frac{\int_{0}^{1} (1-\rho)^6 \rho^5 d\rho}{\int_{0}^{1} (1-\rho)^3 \rho^5 d\rho}$$

Estas integrales son instancias de la función beta, por lo que podemos
evaluar rápidamente el resultado usando scipy:


In [4]:
from scipy.special import beta # Se importa la función beta
P_B_D = beta(6+1, 5+1) / beta(3+1, 5+1) # los valores pueden representar el número de éxitos y fracasos en un contexto de estimación bayesiana
print("La probabilidad de que Bob gane es: ",round(P_B_D,2))

La probabilidad de que Bob gane es:  0.09


* El enfoque bayesiano da probabilidades de 10 a 1 en contra de Bob, mientras que el enfoque frecuentista ingenuo da probabilidades de 18 a 1 en contra de Bob. Entonces, ¿cuál es correcto?
Para un problema tan simple como este, podemos responder a esta pregunta empíricamente simulando una gran cantidad de juegos y contando la fracción de juegos adecuados que Bob gana. El resultado de dicha simulación confirma el resultado
bayesiano: 10 a 1 en contra de la victoria de Bob. ¿El frecuentismo es incorrecto?, no necesariamente: en este caso, el resultado incorrecto es más una cuestión de que el enfoque sea "ingenuo" que de que sea
"frecuentista".

# Confianza vs. Credibilidad

A la hora de construir un límite estándar del 95% sobre un parámetro $\theta$:
 * Un bavesiano diría: "Dados nuestros datos observados, hay un 95% de probabilidad de que el verdadero valor de $\theta$ se encuentre dentro de la región creíble".
 * Un frecuentista diría: "Si este experimento se repite muchas veces, en el 95% de estos casos el intervalo de confianza calculado contendrá el verdadero 0.25".

Observemos la sutil diferencia: el bayesiano enuncia una probabilidad sobre el valor del parámetro dada una región creíble fija. El frecuentista enuncia una probabilidad sobre el intervalo de confianza en sí dado un valor de parámetro fijo. Esta distinción se desprende directamente de la definición de probabilidad que se ha analizado anteriormente, la probabilidad bayesiana es una afirmación del grado de conocimiento sobre un parámetro, la probabilidad frecuentista es una afirmación de la frecuencia límite a largo plazo de las cantidades (como el intervalo de confianza) derivadas de los datos.

El frecuentismo no busca una afirmación probabilística sobre un intervalo fijo, como lo hace el enfoque bayesiano; en cambio, busca afirmaciones probabilísticas sobre un conjunto de intervalos construidos, siendo el intervalo calculado particular sólo una única extracción de entre ellos.

# Ejemplo 3 (Bayesianismo): El método Monte Carlo con cadenas de Markov

Un punto de inflexión en la computación bayesiana práctica fue el desarrollo y la aplicación de métodos de muestreo como Markov Chain Monte Carlo, los cuales son una clase de algoritmos que pueden caracterizar de manera eficiente incluso distribuciones posteriores de alta dimensión mediante la extracción de muestras
aleatorias de manera que los puntos se distribuyan de acuerdo hacia la parte posterior.
A continuación, propondremos vemos modelo sencillo y compararemos un enfoque frecuentista estándar
con tres implementaciones de MCMC disponibles en Python.



### -> Un modelo lineal simple
Consideremos un modelo lineal simple de tres parámetros que ajusta una línea recta a datos con errores desconocidos. Los parámetros serán la intersección con el eje $\alpha$, la pendiente $\beta$ y la dispersión normal (desconocida) o sobre la línea.
Para los datos $D = {(xi, yi)}$, el modelo es

$$ \hat{y}(x_i|\alpha,\beta)=\alpha + \beta x_i$$

y la probabilidad es el producto de la distribución gaussiana para cada
punto:
 $$\mathscr{L}(D|\alpha,\beta,\sigma)=(2\pi\sigma^2)^{-\frac{N}{2}} \prod_{i=1}^{N} \text{ exp }[\frac{-{[y_i-\hat{y}(x_i|\alpha,\beta)]^2}}{2\sigma^2}] $$

 Evaluaremos este modelo en el siguiente conjunto de datos:


In [5]:
np.random.seed(42) # Fija la semilla del generador de números aleatorios para asegurar la reproducibilidad de los resultados
theta_true = (25, 0.5) # Define los parámetros del modelo lineal (intersección y pendiente)
xdata = 100 * np.random.random(20) # Genera 20 valores aleatorios de x entre 0 y 100
ydata = theta_true[0] + theta_true[1] * xdata # Calcula los valores de y correspondientes a la línea recta
ydata = np.random.normal(ydata, 10) # # Agrega ruido aleatorio a los valores de y con una desviación estándar de 10


####-> Solución frecuentista

Se puede encontrar una solución frecuentista calculando la estimación puntual de máxima verosimilitud.
Si definimos el vector $\theta = [\alpha \beta]^T$ el vector de respuesta, $Y=[y_1 , y_2, y_3 , ... , y_N]^T$ y la matriz de diseño:

$$\begin{equation}
\begin{bmatrix}
 1 & 1 & 1 & ... & 1\\
x_1 & x_2 & x_3 & ... & x_N
\end{bmatrix}
\end{equation}$$

Se puede demostrar que la solución de máxima verosimilitud

$$\hat{\theta}=(X^T X )^{-1} (X^T Y) $$

In [6]:
X = np.vstack([np.ones_like(xdata), xdata]).T #Creamos la matriz X de diseño para el modelo de regresión lineal
theta_hat = np.linalg.solve(np.dot(X.T, X),np.dot(X.T, ydata)) #np.dont(..) calcula el producto de matrices
                                                               # np.linalg.solve(...) resuelve el sistema de ecuaciones para obtener theta
y_hat = np.dot(X, theta_hat) # Se predice el valor de y tomando en cuenta los coeficientres obtenidos por theta
sigma_hat = np.std(ydata - y_hat) # Se calcula la desviación estándar de los residuos (errores)
Sigma = sigma_hat ** 2 *  np.linalg.inv(np.dot(X.T, X)) # Se calcula la matriz de covarianza de los estimadores de los coeficientes theta

En la práctica, el enfoque frecuentista suele depender de muchos más diagnósticos estadísticos que la máxima verosimilitud y el intervalo de confianza. Para este problema, se puede utilizar de la siguiente manera:

In [7]:
import statsmodels.api as sm # version 0.5
X = sm.add_constant(xdata) # sm.add_constant(...) agrega un término constante al conjunto de datos de entrada xdata
result = sm.OLS(ydata, X).fit() # sm.OLS(...) crea un modelo de regresión lineal utilizando el metodo de minimos cuadrados
                                # .fit(...) ajusta el modelo a los datos, estimando los coeficientes de la regresión
sigma_hat = result.params    # Contiene los coeficientes estimados (o parámetros) del modelo
Sigma = result.cov_params()   # Contiene la matriz de covarianza de los coeficientes estimados
print(result.summary2())  # Imprime un resumen detallado de los resultados del ajuste del modelo de regresión

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.683   
Dependent Variable: y                AIC:                147.7737
Date:               2024-10-12 02:47 BIC:                149.7651
No. Observations:   20               Log-Likelihood:     -71.887 
Df Model:           1                F-statistic:        41.97   
Df Residuals:       18               Prob (F-statistic): 4.30e-06
R-squared:          0.700            Scale:              86.157  
-------------------------------------------------------------------
            Coef.    Std.Err.     t      P>|t|     [0.025    0.975]
-------------------------------------------------------------------
const      24.6361     3.7871   6.5053   0.0000   16.6797   32.5924
x1          0.4483     0.0692   6.4782   0.0000    0.3029    0.5937
-----------------------------------------------------------------
Omnibus:              1.996        Durbin-Watson:           2.758
Prob(Omnibus):   

Podríamos imaginarnos cómo abordar este problema transformando las varables, por ejemplo, haciendo uso del Prior de Jeffreys en el contexto de un modelo de regresión lineal. Se propone transformar variables utilizando un prior plano en el ángulo que la línea forma con el eje $x$, en lugar de usar la pendiente. A través de transformaciones y usando el Jacobiano, se deduce que la probabilidad conjunta de los parámetros $\alpha$ y $\beta$ debe ser proporcional a $$(1+\beta^2)^{\frac{3}{2}}$$

por lo que se argumenta que para el parámetro $\sigma$ (la desviación estándar), el Prior de Jeffreys es inversamente proporcional a $\sigma$, es decir, $$P(\sigma)∝1/\sigma$$, lo que equivale a un prior plano en el logaritmo de $\sigma$,por lo que el prior no informativo para el problema de regresión lineal es proporcional a $$\frac{1}{\sigma}(1+\beta^2)^{-\frac{3}{2}}$$ y se puede usar este prior junto con la verosimilitud para evaluar el posterior usando métodos como MCMC.



### Solición con presentador

El paquete emcee en Python, implementa un método de muestreo MCMC llamado Affine Invariant Ensemble MCMC. Este método es una versión avanzada de MCMC. El paquete emcee es ligero y fácil de usar, ya que solo requiere definir una función en Python que represente el logaritmo del posterior.

Para mayor claridad, la definición del posterior se descompone en dos partes: el log-prior (logaritmo del prior) y el log-likelihood (logaritmo de la verosimilitud), que se utilizan para el muestreo.

In [8]:
!pip install -q emcee

import emcee # librería para realizar muestreo de cadenas de Markov
import numpy as np # Make sure to import numpy


def log_prior(theta):
    alpha, beta, sigma = theta
    if sigma < 0:
        return -np.inf
    else:
        return (-1.5 * np.log(1 + beta ** 2) - np.log(sigma))

def log_like(theta, x, y):  # Funcion de verosimilitud
    alpha, beta, sigma = theta
    y_model = alpha + beta * x
    # Fix: Use np.pi * sigma instead of np.pisigma
    return -0.5 * np.sum(np.log(2 * np.pi * sigma ** 2) + (y - y_model) ** 2 / sigma ** 2)

def log_posterior(theta, x, y):
    return log_prior(theta) + log_like(theta, x, y)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.2 MB/s eta 0:00:00


A continuación, configuramos el cálculo. emcee combina varios "caminantes" que interactúan, cada uno de los cuales genera su propia cadena de Markov. También especificaremos un período de rodaje para permitir
que las cadenas se estabilicen antes de dibujar los trazos finales:

In [11]:
ndim=3 # Indica que se muestrearán tres parámetros del modelo
nwalkers=50 # Utiliza 50 caminantes para explorar el espacio de parámetros.
nburn=100 # Descartará los primeros 100 pasos de cada caminante
nsteps=20 # Indica que los caminantes realizarán 20 pasos después del "burn-in"
starting_guesses=np.random.rand(nwalkers,ndim) #Los caminantes comienzan en posiciones aleatorias en un cubo tridimensional de valores entre 0 y 1.

In [15]:
sampler=emcee.EnsembleSampler(nwalkers,ndim,log_posterior,args=[xdata,ydata])
sampler.run_mcmc(starting_guesses,nsteps) # se ejecuta el muestreo
# chain is of shape (nwalkers, nsteps, ndim):
# discard burn-in points and reshape:
trace = sampler.chain[:, nburn:, :]
trace = trace.reshape(-1, ndim).T

A continuacion se muestra un paquete que utiliza el clásico muestreador Metropolis-Hastings y ofrece muchas características integradas, como soporte para el muestreo eficiente de distribuciones previas comunes. Debido a estas capacidades, PyMC requiere un poco más de configuración inicial en comparación con emcee, pero resulta ser una herramienta muy poderosa para realizar inferencias bayesianas flexibles.

In [1]:
!pip install -q pymc==4.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 554.5/554.5 kB 5.4 MB/s eta 0:00:00


In [17]:
import pymc # version 2.3

alpha = pymc.Uniform(alpha, -100, 100)
@pymc.stochastic(observed=False)

def beta(value=0):
    return -1.5 * np.log(1 + value**2)

@pymc.stochastic(observed=False)

def sigma(value=1):
    return -np.log(abs(value))

# Define the form of the model and likelihood
@pymc.deterministic

def y_model(x=xdata, alpha=alpha, beta=beta):

   return alpha + beta * x

y = pymc.Normal(y, mu=y_model, tau=1./sigma**2,
observed=True, value=ydata)
# package the full model in a dictionary
model = dict(alpha=alpha, beta=beta, sigma=sigma,y_model=y_model, y=y)



RuntimeError: 
Could not import 'mkl'.  If you are using conda, update the numpy
packages to the latest build otherwise, set MKL_THREADING_LAYER=GNU in
your environment for MKL 2018.

If you have MKL 2017 install and are not in a conda environment you
can set the Aesara flag blas__check_openmp to False.  Be warned that if
you set this flag and don't set the appropriate environment or make
sure you have the right version you *will* get wrong results.


In [9]:
S=pymc.MCMC(model)
S.sample(iter=10000,burn=1000)
trace=[S.trace('alpha')[:],S.trace('beta')[:],S.trace('sigma')[:]]

NameError: name 'pymc' is not defined

### Solución con PyStan
El lenguaje Stan está diseñado especificamente para la expresión de modelos probabilísticos; PyStan permite que los modelos Stan especificados en forma de cadenas de Python sean analizados, compilados y ejecutados por la biblioteca Stan.

In [12]:
!pip install -q pystan
import pystan # version 2.2
model_code = """
data {
int<lower=0> N; // number of points
real x[N]; // x values
real y[N]; // y values
}
parameters {
real alpha_perp;
real<lower=-pi()/2, upper=pi()/2> theta;
real log_sigma;
}
transformed parameters {
real alpha;
real beta;
real sigma;
real ymodel[N];
alpha <- alpha_perp / cos(theta);
beta <- sin(theta);
sigma <- exp(log_sigma);
for (j in 1:N)
ymodel[j] <- alpha + beta * x[j];
}
model {
y ~ normal(ymodel, sigma);
}
"""
# perform the fit & extract traces
data = {N: len(xdata), x: xdata, y: ydata}
fit = pystan.stan(model_code=model_code, data=data,

iter=25000, chains=4)

tr = fit.extract()
trace = [tr[alpha], tr[beta], tr[sigma]]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.9 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'pystan'

Aunque no se logro copilar los códigos que nos mostraba el articulo con los diferentes tipos de soluciones debido a las versiones de las paqueterias que se manejan actualmente, el articulo nos muestra una imagen en la cual se ve el comportamiento de las elipses, por lo que combinando estas ideas y por varios ejemplos realizados anteriormente sobre el uso del frecuentismo y el bayesianismo en un problema de regresión lineal más realista, entonces nos preguntamos ¿Cuál es entonces el mejor enfoque?, esta es una cuestión de ideología personal, pero también depende de la naturaleza del problema en cuestión. Los enfoques frecuentistas suelen calcularse fácilmente y son adecuados para procesos y mediciones verdaderamente repetibles, pero pueden encontrarse con obstáculos con conjuntos pequeños de datos y modelos que se alejan mucho del modelo gaussiano. Existen herramientas frecuentistas para estas situaciones, pero a menudo requieren consideraciones sutiles y conocimientos especializados. Los enfoques bayesianos requieren la especificación de un valor previo potencialmente subjetivo y a menudo implican un cálculo intensivo mediante MCMC.